# Engulf Star Pattern detection

In [165]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [166]:
def engulf_star_pattern(financial_data, fig: go.Figure = None,record_to_plot: int = 2000):
    financial_data = financial_data[:record_to_plot]
    length = len(financial_data)
    highs = list(financial_data['High'])
    lows = list(financial_data['Low'])
    closes = list(financial_data['Close'])
    opens = list(financial_data['Open'])
    bdiffs = [0] * length

    hdiffs = [0] * length
    ldiffs = [0] * length
    r1vals = [0] * length
    r2vals = [0] * length

    def analyzeEngulf(index):
        current = index
        bdiffs[current] = abs(opens[current] - closes[current])

        if bdiffs[current] < 0.000001:
            bdiffs[current] = 0.000001

        bdiff_min = 0.0022

        if (bdiffs[current] > bdiff_min and bdiffs[current - 1] > bdiff_min and
            opens[current - 1] < closes[current - 1] and
            opens[current] > closes[current] and
                (opens[current] - closes[current - 1]) >= -1e-5 and closes[current] < opens[current - 1]):
            return 1, opens[current], closes[current]

        elif (bdiffs[current] > bdiff_min and bdiffs[current - 1] > bdiff_min and
              opens[current - 1] > closes[current - 1] and
              opens[current] < closes[current] and
              (opens[current] - closes[current - 1]) <= 1e-5 and closes[current] > opens[current - 1]):
            return 2,  opens[current], closes[current]

        else:
            return 0, 0, 0

    def identifyStar(index):
        bdiff_min = 0.0022
        current = index

        hdiffs[current] = highs[current] - max(opens[current], closes[current])
        ldiffs[current] = min(opens[current], closes[current]) - lows[current]
        bdiffs[current] = abs(opens[current] - closes[current])

        if bdiffs[current] < 0.000001:
            bdiffs[current] = 0.000001

        r1vals[current] = hdiffs[current] / bdiffs[current]
        r2vals[current] = ldiffs[current] / bdiffs[current]

        if (r1vals[current] > 0.95 and ldiffs[current] < 0.22 * hdiffs[current] and bdiffs[current] > bdiff_min):
            return 1, opens[current], closes[current]

        elif (r2vals[current] > 1.05 and hdiffs[current] < 0.22 * ldiffs[current] and bdiffs[current] > bdiff_min):
            return 2, opens[current], closes[current]

        else:
            return 0, 0, 0

    # activate the functions above to detect engulfing and star patterns on the chart
    results = {"engulfing_1": 0, "engulfing_2": 0, "star_1": 0, "star_2": 0}
    
    # Engulfing Pattern 1: Bullish
    # Engulfing Pattern 2: Bearish
    # Star Pattern 1: Bullish
    # Star Pattern 2: Bearish
    
    
    #add column caleld signal to financial data
    financial_data["signal"] = 0
    
    
    # initialize the figure
    fig = go.Figure()
    

    for i in range(1, length):
        if (analyzeEngulf(i)[0] == 2):
            results["engulfing_1"] += 2
            fig.add_trace(go.Scatter(x=[i], y=[analyzeEngulf(i)[2]], name=f'Engulfing 1', marker=dict(size=12, color="turquoise"), mode="markers"))
            financial_data.loc[i,"signal"] = 2
        elif (analyzeEngulf(i)[0] == 1):
            results["engulfing_2"] += 1
            fig.add_trace(go.Scatter(x=[i], y=[analyzeEngulf(i)[2]], name=f'Engulfing 2', marker=dict(size=12, color="orange"), mode="markers"))
            #financial_data.loc[i,"signal"] = 1
        if (identifyStar(i)[0] == 2):   
            results["star_1"] += 1
            fig.add_trace(go.Scatter(x=[i], y=[identifyStar(i)[2]], name=f'Star 1', marker=dict(size=12, color="pink"), mode="markers"))
            #financial_data.loc[i,"signal"] = 2
        elif (identifyStar(i)[0] == 1):
            results["star_2"] += 1
            fig.add_trace(go.Scatter(x=[i], y=[identifyStar(i)[2]], name=f'Star 2', marker=dict(size=12, color="brown"), mode="markers"))
            financial_data.loc[i,"signal"] = 1

    # plot the closing price
    fig.add_trace(go.Scatter(x=financial_data.index, y=financial_data['Close'], name="Close", line_color='dimgray'))

    return financial_data,fig



In [167]:
financial_data = pd.read_csv("..\data\EURUSD_Candlestick_4_Hour_ASK_05.05.2003-16.10.2021.csv")

# delete dollar sign
try:
    # Clean up dollar signs and convert columns to numeric values
    financial_data["Close/Last"] = financial_data["Close/Last"].str.replace(
        "$", "").astype(float)
    financial_data["Open"] = financial_data["Open"].str.replace(
        "$", "").astype(float)
    financial_data["High"] = financial_data["High"].str.replace(
        "$", "").astype(float)
    financial_data["Low"] = financial_data["Low"].str.replace(
        "$", "").astype(float)
except Exception:
    pass

# rename the headers
try:
    financial_data.rename(columns={"Close/Last": "Close"}, inplace=True)
except Exception:
    pass


In [168]:
financial_data.rename(columns={"high": "High"}, inplace=True)
financial_data.rename(columns={"low": "Low"}, inplace=True)
financial_data.rename(columns={"open": "Open"}, inplace=True)
financial_data.rename(columns={"volume": "Volume"}, inplace=True)
financial_data.rename(columns={"close": "Close"}, inplace=True)


financial_data = financial_data.reset_index(drop=True)

In [169]:
from apply_backtesting import backtester
financial_data,fig = engulf_star_pattern(financial_data, record_to_plot=1000)
bt,stats = backtester(financial_data)
fig.show()

C:\Users\bobek\AppData\Local\Temp\ipykernel_1828\2137152349.py:72: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\bobek\QuantWiseAI\jupiter_notebooks\apply_backtesting.py:34: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [170]:

import backtesting

# import output_notebook
from bokeh.io import output_notebook
output_notebook()

backtesting.set_bokeh_output(notebook=True)
bt.plot()

Loading BokehJS ...

Row(id='18771', ...)

In [1]:
print(stats)

NameError: name 'stats' is not defined